# Calibration the sensor
1. Extrinsics
   - Camera fov on zero surface
   - Camera position
   - LED Position
2. Intrinsics
   - LED Illumination model
   - Camera imaging function (relationship between diffusion intensity and image intensity)  

In [1]:
import mitsuba as mi
import drjit as dr 
import matplotlib.pyplot as plt
mi.set_variant("cuda_ad_rgb")

## Step 1: measure Extrinsics by model

In [2]:
Intrinsics = mi.Matrix3f([[422.3949, 0,313.5413],[0,422.4885,233.1770],[0,0,1]])
Z_camera = -44
image_res = (640,480)
cam_conner1 = dr.inverse(Intrinsics)@mi.Point3f(0,0,1)*-Z_camera
cam_conner2 = dr.inverse(Intrinsics)@mi.Point3f(image_res[0]-1,image_res[1]-1,1)*-Z_camera


# ### Image pixel resolution ###
# image_res = (256,256)
# ### camera origin          ###
cam_origin = mi.Point3f(0,0,Z_camera)
cam_range = ((cam_conner1[0,0],cam_conner1[1,0]),(cam_conner2[0,0],cam_conner2[1,0]))
cam_fov= (cam_conner2[0,0]-cam_conner1[0,0],cam_conner2[1,0]-cam_conner1[1,0])
from utils import camera 
cam = camera.Camera(cam_origin, image_res,cam_range)

## Step 2: fix incident angles, measure diffuse intensity and image intensity at diffrent LED intensity.

In [3]:
import utils.led_ring

N_LED = 24
LED0_pos = mi.Point3f(34.41,0,-12.5)
LED_ring = utils.led_ring.LED_params(N_LED, LED0_pos)

In [4]:
from utils import iristac
sensor = iristac.sensor(cam,LED_ring)

In [5]:
si = cam.shot(sensor.Z0_scene)
P_LED = LED_ring.P
N_LED = LED_ring.N
D_LED = LED_ring.D

# compute incident direction D_incident

P_LED_t = mi.TensorXf(dr.ravel(P_LED),shape=[1,N_LED,1,3])
N_Pixel = image_res[0]*image_res[1]
P_Pixel_t = mi.TensorXf(dr.ravel(si.p),shape=[N_Pixel,1,1,3])
D_incident = P_Pixel_t - P_LED_t
D_incident_n = mi.TensorXf(dr.block_sum(dr.sqr(D_incident.array),3),shape=[N_Pixel,N_LED,1,1])
D_incident = D_incident/dr.sqrt(D_incident_n)

# compute angle between incident and LED normal
D_LED_t = mi.TensorXf(dr.ravel(D_LED),shape=[1,N_LED,1,3])
cosine2 = D_incident*D_LED_t
cosine2 = mi.TensorXf(dr.block_sum(cosine2.array,3),shape=[N_Pixel,N_LED,1,1])
cosine2 = dr.select(cosine2>0,cosine2,0)
I_LED = dr.exp(-1/(cosine2+0.05)/2)*dr.exp(0.5)

theta0_mask =  abs(cosine2-0.5)< 1e-4


In [ ]:

value_dict = {}

for i_LED in [0]:
    D_incident_vec = dr.gather(mi.Vector3f,D_incident[:,i_LED,:,:].array,dr.arange(mi.UInt,N_Pixel))
    i_t = mi.TensorXf(D_incident_vec@si.n,[image_res[1],image_res[0]])
    n_theta = dr.count(theta0_mask[:,i_LED].array)
    i_arr = D_incident_vec@si.n
    phi = dr.exp(-1/(cosine2[:,i_LED].array+0.05)/2)*dr.exp(0.5)
    I_temp = phi*i_arr
    if n_theta==0:
        continue
    for i_RGB in [0,1,2]:
        value_dict[i_LED,i_RGB] = []
        for i_intensity in range(256):
            
            # path = "../data/calibration/xx_xxx_{2:01d}/{0:02d}_{1:03d}_{2:01d}.jpg".format(i_LED,i_intensity,i_RGB)
            path = "../data/calibration_sim/{0:02d}_{1:03d}_{2:01d}.png".format(i_LED,i_intensity,i_RGB)
            bmp1 = mi.Bitmap(path)  
            I_cali = mi.TensorXf(bmp1)/256

            diffuseIntensity_t = i_t*i_intensity/256

            diffuseIntensity = dr.sum(dr.select(theta0_mask[:,i_LED].array,diffuseIntensity_t.array,0))/n_theta
            I_value = dr.sum(dr.select(theta0_mask[:,i_LED].array,I_cali[:,:,i_RGB].array,0))/n_theta

            value_dict[i_LED,i_RGB].append([I_value[0],diffuseIntensity[0]])
            print("\r", end=path)


In [ ]:
import numpy as np

led_list=[0]
rgb_list=[0,1,2]


for i in led_list:
    if not value_dict.keys().__contains__((i,0)):
        continue

    for j in rgb_list:
        plt.scatter(np.array(value_dict[i,j])[:,0],np.array(value_dict[i,j])[:,1])



In [ ]:
k_I = [0,0,0]
for i_RGB in range(3):
    X = []
    Y = []
    for i_LED in [0]:
        X.extend(np.array(value_dict[i_LED,i_RGB])[:,0])
        Y.extend(np.array(value_dict[i_LED,i_RGB])[:,1])
    X = np.array(X)
    Y = np.array(Y)
    k_I[i_RGB] = X.T@Y/(X.T@X)
print(k_I)

## Step 3: fix LED intensity, measure LED angle and compute incident intensity by diffuse intensity.

In [ ]:
i_intensity = 100
i_LED = 16
i_RGB = 2


k_rgb = [0.8,0.7,0.9]

path = "../data/calibration_sim/{0:02d}_{1:03d}_{2:01d}.png".format(i_LED,i_intensity,i_RGB)


D_incident_vec = dr.gather(mi.Vector3f,D_incident[:,i_LED,:].array,dr.arange(mi.UInt,N_Pixel))
i_arr = D_incident_vec@si.n
phi = dr.exp(-1/(cosine2[:,i_LED].array+0.05)/2)*dr.exp(0.5)
I_temp = phi*i_arr*i_intensity/256*k_rgb[i_RGB]

I_sim = mi.TensorXf(0,(480,640,3))
I_sim[:,:,i_RGB] = I_temp

# dr.eval(I_sim1,cosine2)

# mi.util.write_bitmap(path, I_sim2, write_async=False)

bmp2 = mi.Bitmap(path)  
I_cali = mi.TensorXf(bmp2)/256

# dr.eval(I_sim1)
phi2 = I_cali[:,:,i_RGB].array*k_I[i_RGB]/i_arr/i_intensity*256
phi1 = I_sim[:,:,i_RGB].array*k_I[i_RGB]/i_arr/i_intensity*256
# phi3 = I_sim2[:,:,i_RGB].array*k_rgb[i_RGB]/i_arr/i_intensity*256
c_theta = cosine2[:,i_LED].array


plt.scatter(c_theta,phi2)
plt.scatter(c_theta,phi1)
# plt.scatter(c_theta,phi3)
plt.show()

In [ ]:
# dr.max(cosine2)
# a = 12.5/dr.sqrt(cam_range[1][1]**2+12.5**2+(34.41-cam_range[0][0])**2)
# dr.sqrt(1-a**2)
i_arr = D_incident_vec@si.n
cosine2 = dr.select(cosine2>0,cosine2,0)
phi = dr.exp(-1/(cosine2[:,i_LED].array+0.05)/2)*dr.exp(0.5)
I_temp = phi*i_arr*i_intensity/256
I_sim = mi.TensorXf(0,(480,640,3))
I_sim[:,:,i_RGB] = I_temp


path = "../data/calibration_sim/{0:02d}_{1:03d}_{2:01d}.jpg".format(i_LED,i_intensity,i_RGB)
bmp = mi.Bitmap(path)  
I_cali = mi.TensorXf(bmp)/256



f, ax = plt.subplots(1,2)

ax[0].imshow(I_sim)
ax[1].imshow(I_cali)
plt.show()